# Data cleaning

In [0]:
import numpy as np
import pandas as pd
import os

In [0]:
%run ./utils/user_defined_functions

## Utils functions used in notebooks

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
def read_file(year):
    file_path = f"/Volumes/dev_advanced_analytics_hr/default/input_files/hr_turnover/input/employee_data/{year}_DS_v05.xlsx"
    df_y = pd.read_excel(file_path, sheet_name='Report', engine='openpyxl')

    # Rename columns
    df_y.columns = df_y.columns.to_series().apply(simplify_string)
    df_y = df_y.rename(columns={
        'person_id': 'user_sys_id',
        'employment_details_termination_type': 'voluntary_involuntary_turnover',
        'year':'reference_year'
    })

    # Add 'reference_year' column and change data types
    df_y['reference_year'] = year
    df_y['user_sys_id'] = df_y['user_sys_id'].astype(str)
    df_y['hr_manager_other_relationships_user_id'] = df_y['hr_manager_other_relationships_user_id'].astype(str)
    df_y['user_employee_id'] = pd.to_numeric(df_y['user_employee_id'], errors='coerce')
    df_y['salary_review'] = pd.to_numeric(df_y['salary_review'], errors='coerce')
    df_y['check_comp_increase'] = pd.to_numeric(df_y['check_comp_increase'], errors='coerce')
    df_y['check_same_country'] = df_y['check_same_country'].astype(bool)

    # filter on japanese employees
    df_y = df_y[df_y['country_region'] != 'Japan']
    
    df_y['date_of_birth'] = df_y['date_of_birth'].astype(str)

    df_y = df_y.drop(columns=['compensation_changes'])
    print(f"{len(df_y)} rows loaded from year {year}")

    return df_y

In [0]:
def load_data(years):
    """
    Load data starting from the most recent years and concatenate the results obtaining a concatenated DataFrame with all the rows read.
    """
    df = pd.DataFrame()  # DataFrame vuoto per iniziare

    for year in years:
        # Leggi i dati per l'anno corrente
        df_year = read_file(year)  # Funzione read_file deve essere definita separatamente
        
        # Assicurati che la colonna `person_id_vlookup_` sia una stringa
        df_year['person_id_vlookup_'] = df_year['person_id_vlookup_'].astype(str)
        
        # Concatena i dati letti al DataFrame principale
        df = pd.concat([df, df_year], ignore_index=True)

    print("Data load completed:")
    print(f"{len(df)} rows loaded overall")
    
    return df


In [0]:

years = ["2024", "2023", "2022", "2021", "2020"]
df = load_data(years)


6226 rows loaded from year 2024
5236 rows loaded from year 2023
5436 rows loaded from year 2022
4884 rows loaded from year 2021
4825 rows loaded from year 2020
Data load completed:
26607 rows loaded overall


In [0]:
df_expat_in = (
    df[(df['employee_category'] == 'Expat IN') & 
       (df['employee_status'].isin(['Active', 'On Leave']))]
    .assign(expat=True)
)

In [0]:
df_expat_out = (
    df[(df['employee_category'] == 'Expat OUT') & 
       (df['employee_status'] == 'Terminated')]
    .assign(expat=True)
)

In [0]:
df_filtered = (
    df[(df['employee_status'] != 'Dormant') &
       (df['contract_type'] == 'Permanent') &
       (df['employee_category'] == 'Employee')]
    .assign(expat=False)
)

# combine the dataframes
df = pd.concat([df_filtered, df_expat_in, df_expat_out], ignore_index=True)

In [0]:
df

,person_id_vlookup_,user_sys_id,user_employee_id,username,employee_status,salutation,last_name,first_name,middle_name,gender,date_of_birth,place_of_birth,country_of_birth,nationality,second_nationality,marital_status,number_of_children,educational_degree,country_region,legal_entity,employment_details_hire_date,employment_details_initial_entry,employment_details_group_entry,employment_details_termination_date,voluntary_involuntary_turnover,contract_type,employee_category,event,event_reason,event_date,job_title,position,position_entry_date,group_management,unit,department,org_layer,location,cost_center,job_area,job_function,job_family,reference_job,position_class,modified_position_class,global_leveling,modified_global_leveling,fte,employee_weekly_hours,country_weekly_hours,power_of_attorney,sti_campaign,contract_end_date,pay_scale_area,pay_scale_type,local_payroll_id,pay_scale_group,pay_scale_level,hr_manager_other_relationships_user_id,hr_manager_other_relationships_name,manager_user_sys_id,people_manager,company_manager_other_relationships_user_id,company_manager_other_relationships_name,functional_manager_other_relationships_user_id,functional_manager_other_relationships_name,office_email_information_email_address,reference_year,employee_status_aggregated,risk_of_retention,placement,successor,salary_review,check_comp_increase,check_increase_higher_than_inflation,check_same_country,compensation_support_column,compensation_analysis,expat
0,2100133,2100133,2100133,02100133,Active,Mr.,Fleury,Stanislas,NaN,Male,1970-02-13,NANTES,FRA,France,NaN,Married,3,Not Declared,France,FR60-Campari France,2016-07-01,2003-11-17 00:00:00,2016-07-01 00:00:00,NaN,NaN,Permanent,Employee,Position Change,COPO-Change of Position Object,2024-01-01,Plant Director d'Aubevoye,50037653.0,2016-07-01,GSC-Global Supply Chain,SC_EU_ME-Manufacturing & Engineering Europe,50100002-Aubevoye,Local,1047-Aubevoye,SCFR62MF01-SC-FR62-Plant Manag,SC-Supply Chain,SC-ME-Manufacturing & Engineering,SC-ME-ME-Manufacturing & Engineering General,SC-ME-ME04-SM-Manufacturing & Engineering Gene...,57-57,57-57,Senior Management,Senior Management,1.0,35.0,35.0,Yes,STI EMEA,NaN,France,CNVS,NaN,CAD,10A,30001601.0,"Sommerard, Stéphanie",00004610,"Grazzini, Matteo",30005000.0,"De Villiers De La Noue, Geraud",30003987.0,"Loru, Marco",stanislas.fleury@campari.com,2024,Onboard,Low,Rethink,/,NaN,0.015,0.0,True,True,No Salary review/Offcycle lower than inflation,False
1,30001200,30001200,30001200,30001200,Active,NaN,Leger,Patrick,NaN,Male,1969-11-17,Barbezieux,FRA,France,NaN,Civil Union,3,Master's Degree,France,FR60-Campari France,2020-03-01,2020-03-01 00:00:00,2020-03-01 00:00:00,NaN,NaN,Permanent,Employee,Position Change,COPO-Change of Position Object,2024-01-01,Plant Dir Bourg Charente,50037648.0,2020-09-01,GSC-Global Supply Chain,SC_EU_ME-Manufacturing & Engineering Europe,50100003-Bourg Charente,Local,1105-Bourg Charente,SCFR63MF01-SC-FR63-Plant Manag,SC-Supply Chain,SC-ME-Manufacturing & Engineering,SC-ME-ME-Manufacturing & Engineering General,SC-ME-ME04-SM-Manufacturing & Engineering Gene...,57-57,57-57,Senior Management,Senior Management,1.0,35.0,35.0,Yes,STI EMEA,NaN,France,CNVS,NaN,CAD,10A,30001601.0,"Sommerard, Stéphanie",30003987,"Loru, Marco",30003987.0,"Loru, Marco",NaN,NaN,Patrick.Leger@campari.com,2024,Onboard,NaN,NaN,/,NaN,0.030,1.0,True,True,Salary review/Offcycle higher than inflation,False
2,30006707,30006707,30006707,30006707,Active,Mr.,DEPARDON,Augustin,NaN,Male,1969-03-27,Suresnes,FRA,France,NaN,Married,2,Master's Degree,France,FR60-Campari France,2024-06-10,2024-06-10 00:00:00,2024-06-10 00:00:00,NaN,Involuntary,Permanent,Employee,Position Change,COPO-Change of Position Object,2024-07-01,Managing Director House of Cognac and Champagne,60001401.0,2024-06-25,GSM-Global Strategic Marketing,GSM_HCC-Global Strategic Marketing - House of ...,NaN,Global,1014-Paris,CHFR60MS01-Corp-Mktg Structure,SM-Sales & Marketing,SM-MK-Marketing,SM-MK-BM-Brand Management,SM-MK-BM03-EX-Ma

In [0]:
df['employee_status'] = df['employee_status'].replace({'Active': 'onboard', 'On Leave': 'onboard'})
df['employee_status'] = df['employee_status'].where(df['employee_status'] == 'onboard', 'terminated')

In [0]:
df['stay'] = np.where(
    df['employee_status'] == 'onboard', 
    "TRUE", 
    np.where(
        (df['voluntary_involuntary_turnover'] != 'Involuntary') | 
        df['voluntary_involuntary_turnover'].isna(), 
        "TRUE", 
        "FALSE"
    )
)

In [0]:
df = df[df['stay'] == "TRUE"].drop(columns=['voluntary_involuntary_turnover', 'stay'])

In [0]:
# Remove retired employees
df = df[~df['event_reason'].isin([
    "ERT-Early / Voluntary Retirement", 
    "NRT-Normal Retirement", 
    "VOLRET-Voluntary Retirment"
])]

In [0]:
df = df.drop(columns=['event', 'event_reason', 'event_date'])
df = df[~df['global_leveling'].isin(['Blue Collars', 'Senior Executive', 'Officer'])]

In [0]:
# Converts only boolean columns to object
for col in df.select_dtypes(include='bool').columns:
    df[col] = df[col].astype('object')
    
na_values = ["Not available", "not available", "Not Available", "Not Declared", "Undeclared", "Unknown"]

for col in df.columns:
    df[col] = df[col].where(~df[col].isin(na_values), pd.NA)

In [0]:
df

,person_id_vlookup_,user_sys_id,user_employee_id,username,employee_status,salutation,last_name,first_name,middle_name,gender,date_of_birth,place_of_birth,country_of_birth,nationality,second_nationality,marital_status,number_of_children,educational_degree,country_region,legal_entity,employment_details_hire_date,employment_details_initial_entry,employment_details_group_entry,employment_details_termination_date,contract_type,employee_category,job_title,position,position_entry_date,group_management,unit,department,org_layer,location,cost_center,job_area,job_function,job_family,reference_job,position_class,modified_position_class,global_leveling,modified_global_leveling,fte,employee_weekly_hours,country_weekly_hours,power_of_attorney,sti_campaign,contract_end_date,pay_scale_area,pay_scale_type,local_payroll_id,pay_scale_group,pay_scale_level,hr_manager_other_relationships_user_id,hr_manager_other_relationships_name,manager_user_sys_id,people_manager,company_manager_other_relationships_user_id,company_manager_other_relationships_name,functional_manager_other_relationships_user_id,functional_manager_other_relationships_name,office_email_information_email_address,reference_year,employee_status_aggregated,risk_of_retention,placement,successor,salary_review,check_comp_increase,check_increase_higher_than_inflation,check_same_country,compensation_support_column,compensation_analysis,expat
0,2100133,2100133,2100133,02100133,onboard,Mr.,Fleury,Stanislas,NaN,Male,1970-02-13,NANTES,FRA,France,NaN,Married,3,<NA>,France,FR60-Campari France,2016-07-01,2003-11-17 00:00:00,2016-07-01 00:00:00,NaN,Permanent,Employee,Plant Director d'Aubevoye,50037653.0,2016-07-01,GSC-Global Supply Chain,SC_EU_ME-Manufacturing & Engineering Europe,50100002-Aubevoye,Local,1047-Aubevoye,SCFR62MF01-SC-FR62-Plant Manag,SC-Supply Chain,SC-ME-Manufacturing & Engineering,SC-ME-ME-Manufacturing & Engineering General,SC-ME-ME04-SM-Manufacturing & Engineering Gene...,57-57,57-57,Senior Management,Senior Management,1.0,35.0,35.0,Yes,STI EMEA,NaN,France,CNVS,NaN,CAD,10A,30001601.0,"Sommerard, Stéphanie",00004610,"Grazzini, Matteo",30005000.0,"De Villiers De La Noue, Geraud",30003987.0,"Loru, Marco",stanislas.fleury@campari.com,2024,Onboard,Low,Rethink,/,NaN,0.015,0.0,True,True,No Salary review/Offcycle lower than inflation,False
1,30001200,30001200,30001200,30001200,onboard,NaN,Leger,Patrick,NaN,Male,1969-11-17,Barbezieux,FRA,France,NaN,Civil Union,3,Master's Degree,France,FR60-Campari France,2020-03-01,2020-03-01 00:00:00,2020-03-01 00:00:00,NaN,Permanent,Employee,Plant Dir Bourg Charente,50037648.0,2020-09-01,GSC-Global Supply Chain,SC_EU_ME-Manufacturing & Engineering Europe,50100003-Bourg Charente,Local,1105-Bourg Charente,SCFR63MF01-SC-FR63-Plant Manag,SC-Supply Chain,SC-ME-Manufacturing & Engineering,SC-ME-ME-Manufacturing & Engineering General,SC-ME-ME04-SM-Manufacturing & Engineering Gene...,57-57,57-57,Senior Management,Senior Management,1.0,35.0,35.0,Yes,STI EMEA,NaN,France,CNVS,NaN,CAD,10A,30001601.0,"Sommerard, Stéphanie",30003987,"Loru, Marco",30003987.0,"Loru, Marco",NaN,NaN,Patrick.Leger@campari.com,2024,Onboard,NaN,NaN,/,NaN,0.030,1.0,True,True,Salary review/Offcycle higher than inflation,False
2,30006707,30006707,30006707,30006707,onboard,Mr.,DEPARDON,Augustin,NaN,Male,1969-03-27,Suresnes,FRA,France,NaN,Married,2,Master's Degree,France,FR60-Campari France,2024-06-10,2024-06-10 00:00:00,2024-06-10 00:00:00,NaN,Permanent,Employee,Managing Director House of Cognac and Champagne,60001401.0,2024-06-25,GSM-Global Strategic Marketing,GSM_HCC-Global Strategic Marketing - House of ...,NaN,Global,1014-Paris,CHFR60MS01-Corp-Mktg Structure,SM-Sales & Marketing,SM-MK-Marketing,SM-MK-BM-Brand Management,SM-MK-BM03-EX-Marketing Senior Director,60I-60I,60I-60I,Executive,Executive,1.0,35.0,35.0,Yes,STI GROUP FUNCTIONS,NaN,France,CNVS,NaN,CAD,10A,30001601.0,"Sommerard, Stéphanie",30001049,"Villa, Julka",30001049.0,"Villa, Julka",NaN,NaN,augustin.depardon@campari.com,2024,Onboard

In [0]:
df['successor'] = df['successor'].fillna("No")
df['salary_review'] = np.where(df['salary_review'].isna(), "No", "Yes")
df['compensation_analysis'] = df['compensation_analysis'].fillna("no_salary_increase")
df = df.rename(columns={'compensation_analysis': 'cmpnstn_anlys'})
# determine if local or not
df['local'] = np.where(df['nationality'] == df['country_region'], "Yes", "No")
df["today"] = pd.to_datetime(df["reference_year"].astype(int) + 1, format="%Y") 
df["tenure_in_the_position_days"] = (df["today"] - df["position_entry_date"]).dt.days

In [0]:
df['date_of_birth'] = df['date_of_birth'].str.split(' ').str[0]
df['date_of_birth'] = pd.to_datetime(df['date_of_birth'])

df['date_of_birth'] = df['date_of_birth'].dt.date

In [0]:
df['date_of_birth'] = pd.to_datetime(df['date_of_birth'])
df["age"] = df["reference_year"].astype(int) - df["date_of_birth"].dt.year

In [0]:
df['employment_details_initial_entry'] = pd.to_datetime(df['employment_details_initial_entry'], errors='coerce')

df['employment_details_initial_entry'] = df['employment_details_initial_entry'].dt.date

In [0]:
df['employment_details_initial_entry'] = pd.to_datetime(df['employment_details_initial_entry'], errors='coerce')
df['tenure'] = df['reference_year'].astype(int) - df['employment_details_initial_entry'].dt.year

In [0]:
df['tenure_in_the_position_days'] = np.minimum(df['tenure_in_the_position_days'], (df['tenure'] + 1) * 365)
management_levels = ["Management", "Senior Management", "Executive", "Senior Executive", "Officer"]
df['management_non_management'] = np.where(df['global_leveling'].isin(management_levels), "yes", "no")
df['sti_campaign'] = np.where(df['sti_campaign'].isna(), "No", "Yes")
df['position_class_num'] = df['position_class'].str.extract(r'(\d+)').astype(int)
# Identify new hires
df['recently_hired'] = np.where(df['tenure'] <= 3, "Yes", "No")

In [0]:
# Mapping the level of education
graduated_list = [
    "Bachelor's Degree", "Master's Degree", "Post-graduate",
    "Associate's Degree", "Doctoral Degree"
]
not_graduated_list = [
    "Vocational School Diploma", "High School Diploma",
    "Mandatory Secondary Education", "Primary Education"
]
df['education'] = np.where(df['educational_degree'].isin(graduated_list), "Graduated",
                  np.where(df['educational_degree'].isin(not_graduated_list), "Not Graduated", "Unknown"))

In [0]:
# number of placement
placement_mapping = {
    "Low Performer": 1,
    "Watchlist": 2,
    "Good Performer": 3,
    "Strong Performer": 4,
    "Emerging HI-PO": 5,
    "HI-PO": 6
}
df['placement_num'] = df['placement'].map(placement_mapping).fillna(0).astype(int)

In [0]:
# number of global leveling
global_leveling_mapping = {
    "Specialist / Gen Staff": 1,
    "Professional": 2,
    "Senior Professional": 3,
    "Management": 4,
    "Senior Management": 5,
    "Executive": 6
}
df['global_leveling_num'] = df['global_leveling'].map(global_leveling_mapping).fillna(0).astype(int)

In [0]:
df = df.sort_values(['user_sys_id', 'reference_year'])

df['placement_lag1'] = df.groupby('user_sys_id')['placement_num'].shift(1)
df['manager_id_lag1'] = df.groupby('user_sys_id')['manager_user_sys_id'].shift(1)
df['manager_id_lag2'] = df.groupby('user_sys_id')['manager_user_sys_id'].shift(2)
df['global_leveling_num_lag1'] = df.groupby('user_sys_id')['global_leveling_num'].shift(1)
df['global_leveling_num_lag2'] = df.groupby('user_sys_id')['global_leveling_num'].shift(2)
df['global_leveling_num_lag3'] = df.groupby('user_sys_id')['global_leveling_num'].shift(3)
df['job_function_lag1'] = df.groupby('user_sys_id')['job_function'].shift(1)
df['job_function_lag2'] = df.groupby('user_sys_id')['job_function'].shift(2)
df['position_class_num_lag1'] = df.groupby('user_sys_id')['position_class_num'].shift(1)
df['position_class_num_lag2'] = df.groupby('user_sys_id')['position_class_num'].shift(2)
df['position_class_num_lag3'] = df.groupby('user_sys_id')['position_class_num'].shift(3)
df['salary_review_lag1'] = df.groupby('user_sys_id')['salary_review'].shift(1)

In [0]:
df = df.sort_values(by=['user_sys_id', 'reference_year'], ascending=[True, False])
df[df['user_sys_id']=='3'][['placement_num', 'reference_year', 'placement_lag1']]

,placement_num,reference_year,placement_lag1
1464,0,2024,4.0
6736,4,2023,0.0
11190,0,2022,4.0
15685,4,2021,4.0
19843,4,2020,NaN


In [0]:
# calculate 'years since prom'
conditions = [
    (df['position_class_num'] >= df['position_class_num_lag1'] + 2),
    (df['global_leveling_num'] > df['global_leveling_num_lag1']),
    (df['position_class_num'] >= df['position_class_num_lag2'] + 2),
    (df['global_leveling_num'] > df['global_leveling_num_lag2']),
    (df['position_class_num'] >= df['position_class_num_lag3'] + 2),
    (df['global_leveling_num'] > df['global_leveling_num_lag3'])
]

choices = [1, 1, 2, 2, 3, 3]

df['years_since_prom'] = np.select(conditions, choices, default=4)

# calculate 'sal_rev_last_2y'
df['sal_rev_last_2y'] = np.select(
    [(df['salary_review'] == 'Yes') | (df['salary_review_lag1'] == 'Yes')],
    ['Yes'],
    default='No'
)

In [0]:
# calculate 'placement_ev'
placement_conditions = [
    (~pd.isna(df['placement_lag1'])) & (~pd.isna(df['placement_num'])) & (df['placement_lag1'] < df['placement_num']),
    (~pd.isna(df['placement_lag1'])) & (~pd.isna(df['placement_num'])) & (df['placement_lag1'] > df['placement_num'])
]

placement_choices = ['Increased', 'Decreased']

df['placement_ev'] = np.select(placement_conditions, placement_choices, default='Stable')


# calculate 'manager_change'
manager_conditions = [
    (df['manager_id_lag2'] != df['manager_user_sys_id']) & (~pd.isna(df['manager_id_lag2'])) & (~pd.isna(df['manager_user_sys_id'])),
    (df['manager_id_lag1'] != df['manager_user_sys_id']) & (~pd.isna(df['manager_id_lag1'])) & (~pd.isna(df['manager_user_sys_id']))
]

manager_choices = ['Yes', 'Yes']

df['manager_change'] = np.select(manager_conditions, manager_choices, default='No')

# calculate 'job_func_change' 
job_func_conditions = [
    (df['job_function_lag2'] != df['job_function']) & (~pd.isna(df['job_function_lag2'])) & (~pd.isna(df['job_function'])),
    (df['job_function_lag1'] != df['job_function']) & (~pd.isna(df['job_function_lag1'])) & (~pd.isna(df['job_function']))
]

job_func_choices = ['Yes', 'Yes']

df['job_func_change'] = np.select(job_func_conditions, job_func_choices, default='No')

In [0]:
# calculate 'plac_ev_new'
plac_ev_conditions = [
    (~pd.isna(df['placement_num']) & (df['placement_num'] >= 5)),
    (~pd.isna(df['placement_num']) & (df['placement_num'] <= 2)),
    (~pd.isna(df['placement_num']) & ~pd.isna(df['placement_lag1']) & (df['placement_num'] > df['placement_lag1'] + 1)),
    (~pd.isna(df['placement_num']) & ~pd.isna(df['placement_lag1']) & (df['placement_num'] < df['placement_lag1'] - 1))
]
plac_ev_choices = ['Good', 'Bad', 'Good', 'Bad']
df['plac_ev_new'] = np.select(plac_ev_conditions, plac_ev_choices, default='Stable')

In [0]:
# # 1. Calcola 'plac_ev_new' con numpy.select
# plac_ev_conditions = [
#     (df['placement_num'] >= 5),
#     (df['placement_num'] <= 2),
#     (df['placement_num'] > df['placement_lag1'] + 1),
#     (df['placement_num'] < df['placement_lag1'] - 1)
# ]

# plac_ev_choices = ['Good', 'Bad', 'Good', 'Bad']

# df['plac_ev_new'] = np.select(plac_ev_conditions, plac_ev_choices, default='Stable')

# # 2. Calcola 'manager_change' con numpy.select
# manager_conditions = [
#     (df['manager_id_lag2'] != df['manager_user_sys_id']),
#     (df['manager_id_lag1'] != df['manager_user_sys_id'])
# ]

# manager_choices = ['Yes', 'Yes']

# df['manager_change'] = np.select(manager_conditions, manager_choices, default='No')

# # 3. Calcola 'job_func_change' con numpy.select
# job_func_conditions = [
#     (df['job_function_lag2'] != df['job_function']),
#     (df['job_function_lag1'] != df['job_function'])
# ]

# job_func_choices = ['Yes', 'Yes']

# df['job_func_change'] = np.select(job_func_conditions, job_func_choices, default='No')

In [0]:
# countries for each geographic region
countries_semea    = ["France", "Italy", "Spain", "Greece", "South Africa"]
countries_ncee     = ["United Kingdom", "Germany", "Russian Federation", "Belgium", "Switzerland", "Ukraine", "Austria"]
countries_apac     = ["Australia", "Singapore", "Korea (the Republic of)", "China", "India", "New Zealand"]
countries_americas = ["Jamaica", "United States", "Mexico", "Brazil", "Canada", "Argentina", "Peru"]

# create 'geographical_region'
conditions = [
    df['country_region'].isin(countries_semea),  
    df['country_region'].isin(countries_ncee),   
    df['country_region'].isin(countries_apac),  
    df['country_region'].isin(countries_americas)
]

choices = ['Semea', 'Ncee', 'Apac', 'Americas']

df['geographical_region'] = np.select(conditions, choices, default='Unknown')

In [0]:
df['manager_user_sys_id'] = pd.to_numeric(df['manager_user_sys_id'], errors='coerce').astype(str)

wg_promoted_count = df[['manager_user_sys_id', 'reference_year', 'years_since_prom']]

wg_promoted_count = wg_promoted_count[wg_promoted_count['years_since_prom'] < 3]
wg_promoted_count = (
    wg_promoted_count
    .groupby(['manager_user_sys_id', 'reference_year'])
    .size() 
    .reset_index(name='wg_prom_n')
)


In [0]:
wg_onboard_count = (df[df['employee_status'] == 'onboard'] 
                    .groupby(['manager_user_sys_id', 'reference_year'])  
                    .size()  
                    .reset_index(name='wg_onboard_n') 
)
wg_onboard_count

,manager_user_sys_id,reference_year,wg_onboard_n
0,10000102.0,2020,6
1,10000102.0,2021,5
2,10000102.0,2022,4
3,10000102.0,2023,5
4,10000102.0,2024,5
...,...,...,...
4968,91122.0,2024,1
4969,nan,2020,5
4970,nan,2021,3
4971,nan,2022,3


In [0]:
work_group_stats = (df.groupby(['manager_user_sys_id', 'reference_year'])  
                    .agg(wg_n=('manager_user_sys_id', 'size'),  
                         wg_avg_tenure=('tenure', 'mean'))  # mean of tenure
                    .reset_index()  
                   )
work_group_stats


,manager_user_sys_id,reference_year,wg_n,wg_avg_tenure
0,10000102.0,2020,6,6.166667
1,10000102.0,2021,5,7.200000
2,10000102.0,2022,4,9.750000
3,10000102.0,2023,5,9.000000
4,10000102.0,2024,5,10.000000
...,...,...,...,...
5104,91122.0,2024,1,1.000000
5105,nan,2020,5,9.000000
5106,nan,2021,5,12.600000
5107,nan,2022,3,10.000000


In [0]:
df_final = (work_group_stats
            .merge(wg_promoted_count, on=['manager_user_sys_id', 'reference_year'], how='left')
            .merge(wg_onboard_count, on=['manager_user_sys_id', 'reference_year'], how='left')
           )

# Replace NaNs with 0s in count columns
df_final[['wg_prom_n', 'wg_onboard_n']] = df_final[['wg_prom_n', 'wg_onboard_n']].fillna(0)

# calculate percentages
df_final['wg_prom_perc'] = round(df_final['wg_prom_n'] / df_final['wg_n'], 3)
df_final['wg_onboard_perc'] = round(df_final['wg_onboard_n'] / df_final['wg_n'], 3)
df_final['wg_avg_tenure'] = round(df_final['wg_avg_tenure'], 3)

# Nan where wg_n < 3
df_final.loc[df_final['wg_n'] < 3, ['wg_avg_tenure', 'wg_prom_perc', 'wg_onboard_perc']] = np.nan

df_final = df_final[['manager_user_sys_id', 'reference_year', 'wg_n', 'wg_avg_tenure', 'wg_prom_perc', 'wg_onboard_perc']]
df_final

,manager_user_sys_id,reference_year,wg_n,wg_avg_tenure,wg_prom_perc,wg_onboard_perc
0,10000102.0,2020,6,6.167,0.00,1.0
1,10000102.0,2021,5,7.200,0.20,1.0
2,10000102.0,2022,4,9.750,0.25,1.0
3,10000102.0,2023,5,9.000,0.00,1.0
4,10000102.0,2024,5,10.000,0.00,1.0
...,...,...,...,...,...,...
5104,91122.0,2024,1,NaN,NaN,NaN
5105,nan,2020,5,9.000,0.00,1.0
5106,nan,2021,5,12.600,0.20,0.6
5107,nan,2022,3,10.000,0.00,1.0


In [0]:
df = df.merge(df_final, on=["manager_user_sys_id", "reference_year"], how="left")
df

,person_id_vlookup_,user_sys_id,user_employee_id,username,employee_status,salutation,last_name,first_name,middle_name,gender,date_of_birth,place_of_birth,country_of_birth,nationality,second_nationality,marital_status,number_of_children,educational_degree,country_region,legal_entity,employment_details_hire_date,employment_details_initial_entry,employment_details_group_entry,employment_details_termination_date,contract_type,employee_category,job_title,position,position_entry_date,group_management,unit,department,org_layer,location,cost_center,job_area,job_function,job_family,reference_job,position_class,...,check_comp_increase,check_increase_higher_than_inflation,check_same_country,compensation_support_column,cmpnstn_anlys,expat,local,today,tenure_in_the_position_days,age,tenure,management_non_management,position_class_num,recently_hired,education,placement_num,global_leveling_num,placement_lag1,manager_id_lag1,manager_id_lag2,global_leveling_num_lag1,global_leveling_num_lag2,global_leveling_num_lag3,job_function_lag1,job_function_lag2,position_class_num_lag1,position_class_num_lag2,position_class_num_lag3,salary_review_lag1,years_since_prom,sal_rev_last_2y,placement_ev,manager_change,job_func_change,plac_ev_new,geographical_region,wg_n,wg_avg_tenure,wg_prom_perc,wg_onboard_perc
0,10000003,10000003,10000003,10000003,onboard,Ms.,Prosina,Anastasia,NaN,Female,1985-02-24,RUSSIA,RUS,Russian Federation,NaN,<NA>,<NA>,Master's Degree,Russian Federation,RU10-Campari Rus Llc,2011-03-01,2011-03-01,2011-03-01 00:00:00,NaN,Permanent,Employee,SENIOR ACCOUNTANT AP,50017581.0,2011-03-01,BUEUROPE-Business Unit Europe,RUS-Russia,NaN,Local,1005-Moscow,BURU10FI01-RU-Finance,SF-Support Function,SF-FI-Finance,SF-FI-AC-Accounting,SF-FI-AC08-SG-Accounting Specialist/GS,48-48,...,0.045000,0.0,True,True,No Salary review/Offcycle lower than inflation,False,Yes,2025-01-01,5055.0,39.0,13.0,no,48,No,Graduated,0,1,5.0,30001405,30001405,1.0,1.0,1.0,SF-FI-Finance,SF-FI-Finance,48.0,48.0,48.0,Yes,4,Yes,Decreased,No,No,Bad,Ncee,5,9.800,0.2,1.000
1,10000003,10000003,10000003,10000003,onboard,NaN,NaN,NaN,NaN,Female,1985-02-24,RUSSIA,RUS,Russian Federation,NaN,<NA>,<NA>,Master's Degree,Russian Federation,RU10-Campari Rus Llc,2011-03-01,2011-03-01,2011-03-01 00:00:00,NaN,Permanent,Employee,SENIOR ACCOUNTANT AP,50017581.0,2011-03-01,BUNCEE-Business Unit Northern Central Eastern ...,RUS-Russia,NaN,Local,1005-Moscow,BURU10FI01-RU-Finance,SF-Support Function,SF-FI-Finance,SF-FI-AC-Accounting,SF-FI-AC08-SG-Accounting Specialist/GS,48-48,...,0.120000,1.0,True,True,Salary review/Offcycle higher than inflation,False,Yes,2024-01-01,4689.0,38.0,12.0,no,48,No,Graduated,5,1,0.0,30001405,30001405,1.0,1.0,1.0,SF-FI-Finance,SF-FI-Finance,48.0,48.0,48.0,Yes,4,Yes,Increased,No,No,Good,Ncee,5,8.800,0.0,1.000
2,10000003,10000003,10000003,10000003,onboard,NaN,NaN,NaN,NaN,Female,1985-02-24,RUSSIA,RUS,Russian Federation,NaN,<NA>,<NA>,Master's Degree,Russian Federation,RU10-Campari Rus Llc,2011-03-01,2011-03-01,2011-03-01 00:00:00,NaN,Permanent,Employee,SENIOR ACCOUNTANT AP,50017581.0,2011-03-01,BUNCEE-Business Unit Northern Central Eastern ...,RUS-Russia,NaN,Local,1005-Moscow,BURU10FI01-RU-Finance,SF-Support Function,SF-FI-Finance,SF-FI-AC-Accounting,SF-FI-AC08-SG-Accounting Specialist/GS,48-48,...,0.084000,1.0,True,True,Salary review/Offcycle higher than inflation,False,Yes,2023-01-01,4324.0,37.0,11.0,no,48,No,Graduated,0,1,5.0,30001405,30001405,1.0,1.0,NaN,SF-FI-Finance,SF-FI-Finance,48.0,48.0,NaN,Yes,4,Yes,Decreased,No,No,Bad,Ncee,5,7.800,0.0,1.000
3,10000003,10000003,10000003,10000003,onboard,NaN,NaN,NaN,NaN,Female,1985-02-24,RUSSIA,RUS,Russian Federation,NaN,<NA>,<NA>,Master's Degree,Russian Federation,RU10-Campari Rus Llc,2011-03-01,2011-03-01,2011-03-01 00:00:00,NaN,Permanent,Employee,SENIOR ACCOUNTANT AP,50017581.0,2011-03-01,BUNCEE-Business Unit Northern Central Eastern ...,RUS-Russia,NaN,Local,1005-Moscow,CF0103CZ03-BU RU-RU- Accounting,SF-Support Function,SF-FI-Finance,SF-FI-AC

In [0]:
df_manager = df.loc[:, ["user_sys_id", "recently_hired", "reference_year"]].rename(
    columns={"user_sys_id": "manager_user_sys_id", "recently_hired": "mg_rec_hired"}
)

In [0]:
df = df.merge(df_manager, on=["manager_user_sys_id", "reference_year"], how="left")
df["location"] = df["location"].str.replace(r".*-", "", regex=True)

In [0]:
df_check = df.copy()
df_check["children"] = np.select(
    [
        df_check["number_of_children"] == "0", 
        df_check["number_of_children"].isna()  
    ],
    [
        "No",  # assign "No"
        pd.NA  # maintain NaN
    ],
    default="Yes"  # otherwhise assign "Yes"
)

# Replacing "marital_status" values
df_check["marital_status"] = df_check["marital_status"].where(df_check["marital_status"] != 0, "Unknown").fillna("Unknown")


df_check['children']

0        <NA>
1        <NA>
2        <NA>
3        <NA>
4        <NA>
         ... 
17379      No
17380      No
17381      No
17382      No
17383      No
Name: children, Length: 17384, dtype: object

In [0]:
df_check['children'].value_counts()

No     4097
Yes    2921
Name: children, dtype: int64

In [0]:
df = df_check.drop(columns=[
    "company_manager_other_relationships_user_id",
    "company_manager_other_relationships_name",
    "functional_manager_other_relationships_user_id",
    "functional_manager_other_relationships_name",
    "modified_position_class",
    "modified_global_leveling",
    "fte",
    "country_weekly_hours",
    "employee_weekly_hours",
    "hr_manager_other_relationships_user_id",
    "hr_manager_other_relationships_name",
    "check_comp_increase",
    "check_increase_higher_than_inflation",
    "check_same_country",
    "compensation_support_column"
]) 

In [0]:
# Apply the remove_one_value_cols function to remove columns with only one value
df = remove_one_value_cols(df, cols_to_check=df.columns)

No columns removed.


In [0]:
# columns to drop
cols_to_drop = [
    "today",  # only to create tenure_in_pos
    "nationality",  # only to create local
    "educational_degree",  # only to create education
    "second_nationality",
    "person_id_vlookup_",
    "username",
    "user_employee_id",
    "place_of_birth",
    "country_of_birth",
    "department",
    "local_payroll_id",
    "employment_details_hire_date",  # redundant with tenure
    "employment_details_initial_entry",  # tenure has same info
    "employment_details_termination_date",
    # TENURE
    "position_entry_date",  # same info of tenure_in_the_position_days
    "date_of_birth",  # age has same info
    "office_email_information_email_address",  # not interesting
    "contract_end_date",
    "employment_details_group_entry",
    # lag variables to create other variables
    "placement_lag1",
    "manager_id_lag1",
    "manager_id_lag2",
    "global_leveling_num_lag1",
    "global_leveling_num_lag2",
    "global_leveling_num_lag3",
    "job_function_lag1", "job_function_lag2",
    "position_class_num_lag1",
    "position_class_num_lag2",
    "position_class_num_lag3",
    "position_class",
    "placement_num",
    "global_leveling_num",
    "salary_review_lag1",
    "number_of_children",  # substituted by children
    "employee_category",
    "contract_type",
    "employee_status_aggregated",
    "wg_onboard_perc"  
]

df = df.drop(columns=cols_to_drop, errors="ignore")
df


,user_sys_id,employee_status,salutation,last_name,first_name,middle_name,gender,marital_status,country_region,legal_entity,job_title,position,group_management,unit,org_layer,location,cost_center,job_area,job_function,job_family,reference_job,global_leveling,power_of_attorney,sti_campaign,pay_scale_area,pay_scale_type,pay_scale_group,pay_scale_level,manager_user_sys_id,people_manager,reference_year,risk_of_retention,placement,successor,salary_review,cmpnstn_anlys,expat,local,tenure_in_the_position_days,age,tenure,management_non_management,position_class_num,recently_hired,education,years_since_prom,sal_rev_last_2y,placement_ev,manager_change,job_func_change,plac_ev_new,geographical_region,wg_n,wg_avg_tenure,wg_prom_perc,mg_rec_hired,children
0,10000003,onboard,Ms.,Prosina,Anastasia,NaN,Female,Unknown,Russian Federation,RU10-Campari Rus Llc,SENIOR ACCOUNTANT AP,50017581.0,BUEUROPE-Business Unit Europe,RUS-Russia,Local,Moscow,BURU10FI01-RU-Finance,SF-Support Function,SF-FI-Finance,SF-FI-AC-Accounting,SF-FI-AC08-SG-Accounting Specialist/GS,Specialist / Gen Staff,NaN,Yes,NaN,NaN,NaN,NaN,30001405.0,"Chelova, Natalia",2024,Low,Grow in Role,/,No,No Salary review/Offcycle lower than inflation,False,Yes,5055.0,39.0,13.0,no,48,No,Graduated,4,Yes,Decreased,No,No,Bad,Ncee,5,9.800,0.2,NaN,<NA>
1,10000003,onboard,NaN,NaN,NaN,NaN,Female,Unknown,Russian Federation,RU10-Campari Rus Llc,SENIOR ACCOUNTANT AP,50017581.0,BUNCEE-Business Unit Northern Central Eastern ...,RUS-Russia,Local,Moscow,BURU10FI01-RU-Finance,SF-Support Function,SF-FI-Finance,SF-FI-AC-Accounting,SF-FI-AC08-SG-Accounting Specialist/GS,Specialist / Gen Staff,NaN,No,NaN,NaN,NaN,NaN,30001405.0,"Chelova, Natalia",2023,Low,Emerging HI-PO,not significant,Yes,Salary review/Offcycle higher than inflation,False,Yes,4689.0,38.0,12.0,no,48,No,Graduated,4,Yes,Increased,No,No,Good,Ncee,5,8.800,0.0,NaN,<NA>
2,10000003,onboard,NaN,NaN,NaN,NaN,Female,Unknown,Russian Federation,RU10-Campari Rus Llc,SENIOR ACCOUNTANT AP,50017581.0,BUNCEE-Business Unit Northern Central Eastern ...,RUS-Russia,Local,Moscow,BURU10FI01-RU-Finance,SF-Support Function,SF-FI-Finance,SF-FI-AC-Accounting,SF-FI-AC08-SG-Accounting Specialist/GS,Specialist / Gen Staff,NaN,No,NaN,NaN,NaN,NaN,30001405.0,"Chelova, Natalia",2022,Low,NaN,No,Yes,Salary review/Offcycle higher than inflation,False,Yes,4324.0,37.0,11.0,no,48,No,Graduated,4,Yes,Decreased,No,No,Bad,Ncee,5,7.800,0.0,NaN,<NA>
3,10000003,onboard,NaN,NaN,NaN,NaN,Female,Unknown,Russian Federation,RU10-Campari Rus Llc,SENIOR ACCOUNTANT AP,50017581.0,BUNCEE-Business Unit Northern Central Eastern ...,RUS-Russia,Local,Moscow,CF0103CZ03-BU RU-RU- Accounting,SF-Support Function,SF-FI-Finance,SF-FI-AC-Accounting,SF-FI-AC08-SG-Accounting Junior Specialist,Specialist / Gen Staff,NaN,No,NaN,NaN,NaN,NaN,30001405.0,"Chelova, Natalia",2021,Low,Emerging HI-PO,No,Yes,Salary review/Offcycle higher than inflation,False,Yes,3959.0,36.0,10.0,no,48,No,Graduated,4,Yes,Increased,No,No,Good,Ncee,6,7.333,0.0,NaN,<NA>
4,10000003,onboard,NaN,NaN,NaN,NaN,Female,Unknown,Russian Federation,RU10-Campari Rus Llc,SENIOR ACCOUNTANT AP,50017581.0,BUNCEE-Business Unit Northern Central Eastern ...,RUS-Russia,Local,Moscow,CF0103CZ03-BU RU-RU- Accounting,SF-Support Function,SF-FI-Finance,SF-FI-AC-Accounting,SF-FI-AC08-SG-Accounting Junior Specialist,Specialist / Gen Staff,NaN,No,NaN,NaN,NaN,NaN,30001405.0,"Chelova, Natalia",2020,Low,Good Performer,No,Yes,Salary review/Offcycle higher than inflation,False,Yes,3594.0,35.0,9.0,no,48,No,Graduated,4,Yes,Stable,No,No,Stable,Ncee,5,6.600,0.0,NaN,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17379,91908,onboard,NaN,Riello,Simone,NaN,Male,Unknown,Italy,IT10-Davide Campari Milano N.V.,Rare Specialist Area 1,90000177.0,BUEUROPE-Business Unit Europe,ITA-Italy,Local,REMOTE,BUIT10SS02-I

## Obtaining train and test data

In [0]:
def get_train_data(df, current_year):
    # Filter to delete data for the current year
    df = df[df['reference_year'] != current_year]

    # Filter "terminated" employees (left the company)
    leavers = df[df['employee_status'] == 'terminated'] \
        .groupby('user_sys_id') \
        .apply(lambda x: x.sort_values('reference_year').iloc[0]) \
        .reset_index(drop=True)

    # Filter "onboard" (hired) employees
    onboard = df[df['employee_status'] == 'onboard'] \
        .groupby('user_sys_id') \
        .apply(lambda x: x.sort_values('reference_year', ascending=False).iloc[0]) \
        .reset_index(drop=True)

    # Removes "onboard" employees who are also "leavers"
    onboard_not_terminated = onboard[~onboard['user_sys_id'].isin(leavers['user_sys_id'])]

    # Merges "onboard" employees who are not "leavers" with "leavers"
    df = pd.concat([onboard_not_terminated, leavers])

    return df

In [0]:
current_year = "2024"

# Apply get_train_data function to select only last year's onboarded employees
df_train = get_train_data(df, current_year) \
    .drop_duplicates()

# Select current year data for prediction
df_predict = df[df['reference_year'] == current_year] \
    .drop_duplicates()

In [0]:
cols_too_many_na = df_train.columns[df_train.isna().mean() >= 0.3].to_list()

# removing columns
cols_too_many_na = list(set(cols_too_many_na) - set([
    'voluntary_involuntary_turnover',
    'placement',
    'successor',
    'salary_review',
    'sti_campaign'
]))

cols_too_many_na

['mg_rec_hired',
 'pay_scale_level',
 'pay_scale_group',
 'risk_of_retention',
 'first_name',
 'middle_name',
 'last_name',
 'children',
 'pay_scale_type',
 'pay_scale_area',
 'salutation',
 'power_of_attorney']

In [0]:
df_train = df_train.drop(columns=cols_too_many_na)
df_predict = df_predict.drop(columns=cols_too_many_na)

In [0]:
df_train.to_csv(f"/Volumes/dev_advanced_analytics_hr/default/input_files/hr_turnover/data_cleaning/data_clean_train_v06.csv", sep=";", index=False)

In [0]:
df_predict.to_csv(f"/Volumes/dev_advanced_analytics_hr/default/input_files/hr_turnover/data_cleaning//data_clean_predict_v06.csv", sep=";", index=False)